# Simulate a small company

b) Use gemini to simulate 20 data points in json format containing the following fields: first_name, last_name, phone_number, email, department, salary, title. See if you can prompt to direct the LLM output to have swedish names, phone numbers in swedish format (+46 731 29 52), departments (IT, HR, marketing, sales), reasonable salary (you might need to check some swedish statistics on salaries) and corresponding titles within these departments.

In [4]:
from google import genai

client = genai.Client()

response = client.models.generate_content(
    model= "gemini-2.5-flash",
    contents= """Give me 20 different data of empoyee in a company with json fromat. 
    It should contain follow fields: 
    first_name, last_name, phone_number, email, department, salary, title. 
    Name, phone and salar should be adapted to Swedish environment"""
)


response.text






'```json\n[\n  {\n    "first_name": "Erik",\n    "last_name": "Andersson",\n    "phone_number": "070-123 45 67",\n    "email": "erik.andersson@examplecompany.se",\n    "department": "Engineering",\n    "salary": 55000,\n    "title": "Senior Software Developer"\n  },\n  {\n    "first_name": "Anna",\n    "last_name": "Johansson",\n    "phone_number": "072-987 65 43",\n    "email": "anna.johansson@examplecompany.se",\n    "department": "Marketing",\n    "salary": 48000,\n    "title": "Marketing Specialist"\n  },\n  {\n    "first_name": "Johan",\n    "last_name": "Karlsson",\n    "phone_number": "073-555 11 22",\n    "email": "johan.karlsson@examplecompany.se",\n    "department": "Sales",\n    "salary": 62000,\n    "title": "Sales Manager"\n  },\n  {\n    "first_name": "Sara",\n    "last_name": "Nilsson",\n    "phone_number": "070-222 33 44",\n    "email": "sara.nilsson@examplecompany.se",\n    "department": "Human Resources",\n    "salary": 52000,\n    "title": "HR Business Partner"\n  },

c) Now use pydantic to validate this json and put in proper schema that the fields should follow. You might need to do some processing such as removing backticks and maybe loading json data into a list with json.loads(). Also make sure that only correctly validated data should be stored.

In [12]:
from pydantic import BaseModel, Field, EmailStr
from typing import Literal
# import phonenumbers

class Employee(BaseModel):
    first_name: str = Field(description="Swedish first name")
    last_name: str = Field(description= "Swedish last name")
    phone_number: str = Field(description="Swedish phone number, e.g +46705435647")
    email: EmailStr
    department: Literal["IT", "HR", "marketing", "sales"] 
    salary: float = Field(description="Salary in swedish currentcy and adapted to Swedish marketing")







d) Write this json data to a folder called output_data.

In [32]:
from pathlib import Path
from google import genai

path_output_data = Path.cwd()/"output_data"

client = genai.Client()

prompt = """Give me 20 different data of empoyee in a company with json fromat. 
    It should contain follow fields: 
    first_name, last_name, phone_number, email, department, salary, title."""

response = client.models.generate_content(
    model= "gemini-2.5-flash",
    contents= prompt,
    config = {
        "response_mime_type": "application/json",
        "response_schema": list[Employee]
    }
)


response.text


employee_data = response.text

In [33]:
import os

# check if folder exists, if not create one
os.makedirs("output_data", exist_ok= True)


with open("output_data/employee.json", "w", encoding= "utf-8") as file:
    
    file.write(employee_data)





e) Use pandas to read the data as dataframe

In [ ]:
import pandas as pd

employee = pd.read_json("output_data/employee.json")


df = pd.DataFrame(employee)

df

,first_name,last_name,phone_number,email,department,salary
0,Anna,Andersson,46701234567,anna.andersson@example.com,IT,45000
1,Erik,Johansson,46702345678,erik.johansson@example.com,sales,48000
2,Sara,Karlsson,46703456789,sara.karlsson@example.com,marketing,42000
3,Per,Nilsson,46704567890,per.nilsson@example.com,HR,40000
4,Maria,Lindgren,46705678901,maria.lindgren@example.com,IT,50000
5,Jonas,Svensson,46706789012,jonas.svensson@example.com,sales,52000
6,Emma,Berg,46707890123,emma.berg@example.com,marketing,43000
7,Daniel,Pettersson,46708901234,daniel.pettersson@example.com,HR,41000
8,Linda,Gustafsson,46709012345,linda.gustafsson@example.com,IT,47000
9,Martin,Holm,46700123456,martin.holm@example.com,sales,49000


f) Write a csv file to your output_data

In [ ]:


df.to_csv("output_data/employee.csv", index=False, encoding= "utf-8")



g) Load this data into a staging layer and store this into a table called employees.

In [46]:
import dlt 
from pathlib import Path


path_duckdb = str(Path.cwd()/"data_warehouse/employees.duckdb")


pipeline = dlt.pipeline(
    pipeline_name= "staff",
    destination= dlt.destinations.duckdb(path_duckdb),
    dataset_name= "staging"
)

employee_dict = df.to_dict(orient="records")

info = pipeline.run(data= employee_dict, table_name= "employees")
    
    
print(info)
    
    




Pipeline staff load step completed in 0.20 seconds
1 load package(s) were loaded to destination duckdb and into dataset staging
The duckdb destination used duckdb:///c:\Users\MAER\Documents\STI\dataplattform_maskininlärning_artificiell_intelligens\AI_engineering_Marcus_Ericsson_de24\exercises\exercise_0\4_simulate_small_company\data_warehouse\employees.duckdb location to store data
Load package 1764456864.1894236 is LOADED and contains no failed jobs
